# Rule Generator (Optimisation algorithm) Example

This notebook contains an example of how the Rule Generator (Optimisation algorithm) can be used to create rules based on a labelled dataset. This algorithm generate rules by optimising the thresholds of single features and combining these one condition rules with AND conditions to create more complex rules.

## Requirements

To run, you'll need the following:

* Install the Rule Generator (Optimisation algorithm) package - see the readme for more information.
* A labelled, processed dataset (i.e. nulls imputed, categorical features one hot encoded, feature selection applied)

----

## Import packages

In [1]:
from rule_generation.rule_generator_opt import RuleGeneratorOpt

import random
import pandas as pd
import numpy as np

## Read in data

Let's read in some labelled, processed dummy data.

In [2]:
X_train = pd.read_csv('dummy_data/X_train.csv', index_col='eid')
y_train = pd.read_csv('dummy_data/y_train.csv', index_col='eid').squeeze()
X_test = pd.read_csv('dummy_data/X_test.csv', index_col='eid')
y_test = pd.read_csv('dummy_data/y_test.csv', index_col='eid').squeeze()

----

## Generate rules

### Set up class parameters

Now we can set our class parameters for the Rule Generator. Here we're using the F1 score as the optimisation function (you can choose a different function from the optimisation_functions module or create your own). 

**Note that if you're using the FScore, Precision or Recall score as the optimisation function, use the *FScore*, *Precision* or *Recall* classes in the *optimisation_functions* module rather than the same functions from Sklearn's *metrics* module, as the former are ~100 times faster on larger datasets.**

**Please see the class docstring for more information on each parameter.**

In [3]:
from rule_optimisation.optimisation_functions import FScore
fs = FScore(beta=1)

In [4]:
params = {
    'opt_func': fs.fit,
    'n_total_conditions': 4,
    'num_rules_keep': 50,
    'n_points': 10,
    'ratio_window': 2,
    'remove_corr_rules': False
}

### Instantiate class and run fit method

Once the parameters have been set, we can run the *.fit()* method to generate rules.

In [5]:
rg = RuleGeneratorOpt(**params)
X_rules = rg.fit(X=X_train, y=y_train, sample_weight=None)

### Outputs

The *.fit()* method returns a dataframe giving the binary columns of the generated rules as applied to the training dataset.

Useful attributes created by running the *.fit()* method are:

* rule_descriptions: A dataframe showing the logic of the generated rules and their performance metrics as applied to the training dataset.
* rules: Class containing the rule stored in the standard ARGO string format. Methods from this class can be used to convert the rules into the standard ARGO dictionary or lambda expression representations. See the *rules* module for more information.

In [6]:
X_rules.head()

,RGO_Rule_20210226_0,RGO_Rule_20210226_1,RGO_Rule_20210226_2,RGO_Rule_20210226_102,RGO_Rule_20210226_101,RGO_Rule_20210226_151,RGO_Rule_20210226_103,RGO_Rule_20210226_150,RGO_Rule_20210226_149,RGO_Rule_20210226_5,...,RGO_Rule_20210226_359,RGO_Rule_20210226_601,RGO_Rule_20210226_637,RGO_Rule_20210226_672,RGO_Rule_20210226_706,RGO_Rule_20210226_317,RGO_Rule_20210226_441,RGO_Rule_20210226_440,RGO_Rule_20210226_360,RGO_Rule_20210226_316
eid,,,,,,,,,,,,,,,,,,,,,
867-8837095-9305559,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
974-5306287-3527394,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
584-0112844-9158928,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
956-4190732-7014837,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
349-7005645-8862067,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
rg.rule_descriptions.head()

,Precision,Recall,PercDataFlagged,OptMetric,Logic,nConditions
Rule,,,,,,
RGO_Rule_20210226_0,0.991837,1.000000,0.027547,0.995902,(X['account_number_num_fraud_transactions_per_...,1
RGO_Rule_20210226_1,0.991837,1.000000,0.027547,0.995902,(X['account_number_num_fraud_transactions_per_...,1
RGO_Rule_20210226_2,0.972000,1.000000,0.028109,0.985801,(X['account_number_num_fraud_transactions_per_...,1
RGO_Rule_20210226_102,0.995781,0.971193,0.026647,0.983333,(X['account_number_num_fraud_transactions_per_...,2
RGO_Rule_20210226_101,0.995781,0.971193,0.026647,0.983333,(X['account_number_num_fraud_transactions_per_...,2


----

## Apply rules to a separate dataset

Use the *.apply()* method to apply the generated rules to a separate dataset.

In [8]:
X_rules_applied = rg.apply(X=X_test, y=y_test, sample_weight=None)

### Outputs

The *.apply()* method returns a dataframe giving the binary columns of the rules as applied to the given dataset.

A useful attribute created by running the *.apply()* method is:

* rule_descriptions_applied: A dataframe showing the logic of the generated rules and their performance metrics as applied to the given dataset.

In [9]:
rg.rule_descriptions_applied.head()

,Precision,Recall,PercDataFlagged,OptMetric,Logic,nConditions
Rule,,,,,,
RGO_Rule_20210226_0,0.991453,1.000000,0.026700,0.995708,(X['account_number_num_fraud_transactions_per_...,1
RGO_Rule_20210226_1,0.991453,1.000000,0.026700,0.995708,(X['account_number_num_fraud_transactions_per_...,1
RGO_Rule_20210226_2,0.958678,1.000000,0.027613,0.978903,(X['account_number_num_fraud_transactions_per_...,1
RGO_Rule_20210226_102,1.000000,0.956897,0.025331,0.977974,(X['account_number_num_fraud_transactions_per_...,2
RGO_Rule_20210226_101,1.000000,0.956897,0.025331,0.977974,(X['account_number_num_fraud_transactions_per_...,2


In [10]:
X_rules_applied.head()

Rule,RGO_Rule_20210226_0,RGO_Rule_20210226_1,RGO_Rule_20210226_2,RGO_Rule_20210226_102,RGO_Rule_20210226_101,RGO_Rule_20210226_151,RGO_Rule_20210226_103,RGO_Rule_20210226_150,RGO_Rule_20210226_149,RGO_Rule_20210226_3,...,RGO_Rule_20210226_442,RGO_Rule_20210226_544,RGO_Rule_20210226_487,RGO_Rule_20210226_482,RGO_Rule_20210226_388,RGO_Rule_20210226_523,RGO_Rule_20210226_522,RGO_Rule_20210226_521,RGO_Rule_20210226_447,RGO_Rule_20210226_316
eid,,,,,,,,,,,,,,,,,,,,,
975-8351797-7122581,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
785-6259585-7858053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
057-4039373-1790681,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
095-5263240-3834186,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
980-3802574-0009480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


----

## The End

That's it folks - if you have any queries or suggestions please put them in the *#sim-datatools-help* Slack channel or email James directly.